In [601]:
import pandas as pd

In [602]:
df = pd.read_csv("nba_games.csv", index_col=0)

In [603]:
df = df.sort_values("date")

In [604]:
df = df.reset_index(drop = True)

In [605]:
del df["mp.1"]
del df["mp_opp.1"]
del df["index_opp"]

In [606]:
def add_target(team):
    team["target"] = team["won"].shift(-1)
    return team

df = df.groupby("team" , group_keys=False).apply(add_target)

/var/folders/yj/39zcm1_x6rx85fq2f58zgx180000gn/T/ipykernel_21841/3382585029.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/yj/39zcm1_x6rx85fq2f58zgx180000gn/T/ipykernel_21841/3382585029.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  team["target"] = team["won"].shift(-1)
/var/folders/yj/39zcm1_x6rx85fq2f58zgx180000gn/T/ipykernel_21841/3382585029.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, 

In [607]:
df.loc[pd.isnull(df["target"]) , "target"] = 2

In [608]:
df["target"]  = df["target"].astype(int, errors = "ignore")

In [609]:
df["won"].value_counts()

won
False    7588
True     7588
Name: count, dtype: int64

In [610]:
df["target"].value_counts()

target
1    7574
0    7572
2      30
Name: count, dtype: int64

In [611]:
nulls = pd.isnull(df)

In [612]:
nulls = nulls.sum()

In [613]:
nulls = nulls[nulls>0]

In [614]:
valid_columns = df.columns[~df.columns.isin(nulls.index)]

In [615]:
df = df[valid_columns].copy()

In [616]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import RidgeClassifier

rr = RidgeClassifier(alpha=1)
split = TimeSeriesSplit(n_splits = 3)

sfs = SequentialFeatureSelector(rr, n_features_to_select=30, direction = "forward", cv=split)

In [617]:
removed_columns = ["season", "date" , "won" , "target" , "team", "team_opp", "total", "total_opp" ]

In [618]:
selected_columns = df.columns[~df.columns.isin(removed_columns)]

In [619]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])

In [620]:
sfs.fit(df[selected_columns],  df["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [621]:
predictors = list(selected_columns[sfs.get_support()])

In [622]:
def backtest(data, model, predictors, start=2, step=1):
    all_predictions = []

    seasons = sorted(data["season"].unique())

    for i in range(start, len(seasons), step):
        season = seasons[i]

        train = data[data["season"] < season]
        test = data[data["season"] == season]

        model.fit(train[predictors], train["target"])

        preds = model.predict(test[predictors])
        preds = pd.Series(preds, index = test.index)

        combined = pd.concat([test["target"], preds], axis = 1)
        combined.columns = ["actual",  "prediction"]

        all_predictions.append(combined)
    return pd.concat(all_predictions)

In [623]:
predictions = backtest(df, rr, predictors)

In [624]:
from sklearn.metrics import accuracy_score

predictions = predictions[predictions["actual"] !=2]

accuracy_score(predictions["actual"], predictions["prediction"])

0.5439624853458382

In [625]:
df.groupby("home").apply(lambda x: x[x["won"] ==1].shape[0] / x.shape[0] , include_groups=False)

home
0.0    0.439378
1.0    0.560622
dtype: float64

In [626]:
df_rolling = df[list(selected_columns) + ["won", "team", "season"]]

In [627]:
def find_team_averages(team):
    rolling = team[selected_columns].rolling(15).mean()
    return rolling

df_rolling = df_rolling.groupby(["team", "season"], group_keys= False).apply(find_team_averages, include_groups=False)

In [628]:
rolling_cols = [f"{col}_15" for col in df_rolling.columns]
df_rolling.columns = rolling_cols

df = pd.concat([df, df_rolling], axis = 1)

In [629]:
df = df.dropna()

In [630]:
def shift_col(team,col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("team", group_keys = False).apply(lambda x: shift_col(x, col_name) , include_groups = False)

df["home_next"] = add_col(df, "home")
df["team_opp_next"] = add_col(df, "team_opp")
df["date_next"] = add_col(df, "date")

In [631]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,orb,...,stl%_max_opp_15,blk%_max_opp_15,tov%_max_opp_15,usg%_max_opp_15,ortg_max_opp_15,drtg_max_opp_15,home_opp_15,home_next,team_opp_next,date_next
392,0.0,0.409091,0.327869,0.480861,0.481481,0.433333,0.450194,0.704545,0.722222,0.344828,...,0.043533,0.084200,0.338365,0.227179,0.305724,0.441667,0.533333,0.0,ORL,2018-11-14
397,0.0,0.409091,0.459016,0.382775,0.370370,0.283333,0.485123,0.454545,0.388889,0.448276,...,0.052200,0.077200,0.478127,0.128120,0.360943,0.554545,0.400000,0.0,HOU,2018-11-15
408,0.0,0.454545,0.377049,0.497608,0.444444,0.316667,0.535576,0.227273,0.277778,0.344828,...,0.042200,0.084133,0.335500,0.212735,0.300337,0.465152,0.533333,1.0,UTA,2018-11-16
410,0.0,0.363636,0.442623,0.339713,0.444444,0.366667,0.477361,0.431818,0.444444,0.379310,...,0.055867,0.083533,0.349057,0.123248,0.343771,0.453030,0.533333,1.0,POR,2018-11-16
412,0.0,0.545455,0.475410,0.526316,0.074074,0.166667,0.169470,0.227273,0.314815,0.413793,...,0.053800,0.081400,0.375262,0.114872,0.346465,0.514394,0.400000,0.0,BOS,2018-11-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15171,0.0,0.386364,0.311475,0.464115,0.555556,0.600000,0.389392,0.295455,0.259259,0.206897,...,0.039400,0.084067,0.461076,0.168462,0.411111,0.572727,0.466667,0.0,DAL,2024-06-14
15172,0.0,0.181818,0.278689,0.224880,0.444444,0.516667,0.351876,0.272727,0.240741,0.137931,...,0.039400,0.090267,0.461076,0.176068,0.412121,0.550000,0.466667,1.0,DAL,2024-06-17
15173,0.0,0.568182,0.459016,0.564593,0.481481,0.450000,0.434670,0.340909,0.407407,0.448276,...,0.050800,0.089800,0.248358,0.214188,0.407744,0.533333,0.533333,0.0,BOS,2024-06-17
15174,0.0,0.386364,0.426230,0.377990,0.407407,0.483333,0.341527,0.386364,0.370370,0.517241,...,0.039800,0.090533,0.463242,0.197863,0.433670,0.559848,0.400000,NaN,None,None


In [632]:
df.loc[15174, "home_next"] = 1.0
df.loc[15175, "home_next"] = 0.0
df.loc[15174, "team_opp_next"] = "NYK"
df.loc[15175, "team_opp_next"] = "BOS"
df.loc[15174, "date_next"] = "2024-10-22"
df.loc[15175, "date_next"] = "2024-10-22"

In [633]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,orb,...,stl%_max_opp_15,blk%_max_opp_15,tov%_max_opp_15,usg%_max_opp_15,ortg_max_opp_15,drtg_max_opp_15,home_opp_15,home_next,team_opp_next,date_next
392,0.0,0.409091,0.327869,0.480861,0.481481,0.433333,0.450194,0.704545,0.722222,0.344828,...,0.043533,0.084200,0.338365,0.227179,0.305724,0.441667,0.533333,0.0,ORL,2018-11-14
397,0.0,0.409091,0.459016,0.382775,0.370370,0.283333,0.485123,0.454545,0.388889,0.448276,...,0.052200,0.077200,0.478127,0.128120,0.360943,0.554545,0.400000,0.0,HOU,2018-11-15
408,0.0,0.454545,0.377049,0.497608,0.444444,0.316667,0.535576,0.227273,0.277778,0.344828,...,0.042200,0.084133,0.335500,0.212735,0.300337,0.465152,0.533333,1.0,UTA,2018-11-16
410,0.0,0.363636,0.442623,0.339713,0.444444,0.366667,0.477361,0.431818,0.444444,0.379310,...,0.055867,0.083533,0.349057,0.123248,0.343771,0.453030,0.533333,1.0,POR,2018-11-16
412,0.0,0.545455,0.475410,0.526316,0.074074,0.166667,0.169470,0.227273,0.314815,0.413793,...,0.053800,0.081400,0.375262,0.114872,0.346465,0.514394,0.400000,0.0,BOS,2018-11-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15171,0.0,0.386364,0.311475,0.464115,0.555556,0.600000,0.389392,0.295455,0.259259,0.206897,...,0.039400,0.084067,0.461076,0.168462,0.411111,0.572727,0.466667,0.0,DAL,2024-06-14
15172,0.0,0.181818,0.278689,0.224880,0.444444,0.516667,0.351876,0.272727,0.240741,0.137931,...,0.039400,0.090267,0.461076,0.176068,0.412121,0.550000,0.466667,1.0,DAL,2024-06-17
15173,0.0,0.568182,0.459016,0.564593,0.481481,0.450000,0.434670,0.340909,0.407407,0.448276,...,0.050800,0.089800,0.248358,0.214188,0.407744,0.533333,0.533333,0.0,BOS,2024-06-17
15174,0.0,0.386364,0.426230,0.377990,0.407407,0.483333,0.341527,0.386364,0.370370,0.517241,...,0.039800,0.090533,0.463242,0.197863,0.433670,0.559848,0.400000,1.0,NYK,2024-10-22


In [634]:
df = df.copy()

In [635]:
full = df.merge(df[rolling_cols + ["team_opp_next", "date_next", "team"]],left_on=["team", "date_next"], right_on = ["team_opp_next", "date_next"])

In [636]:
removed_columns = list(full.columns[full.dtypes == "object"]) + removed_columns

In [637]:
df

,mp,fg,fga,fg%,3p,3pa,3p%,ft,fta,orb,...,stl%_max_opp_15,blk%_max_opp_15,tov%_max_opp_15,usg%_max_opp_15,ortg_max_opp_15,drtg_max_opp_15,home_opp_15,home_next,team_opp_next,date_next
392,0.0,0.409091,0.327869,0.480861,0.481481,0.433333,0.450194,0.704545,0.722222,0.344828,...,0.043533,0.084200,0.338365,0.227179,0.305724,0.441667,0.533333,0.0,ORL,2018-11-14
397,0.0,0.409091,0.459016,0.382775,0.370370,0.283333,0.485123,0.454545,0.388889,0.448276,...,0.052200,0.077200,0.478127,0.128120,0.360943,0.554545,0.400000,0.0,HOU,2018-11-15
408,0.0,0.454545,0.377049,0.497608,0.444444,0.316667,0.535576,0.227273,0.277778,0.344828,...,0.042200,0.084133,0.335500,0.212735,0.300337,0.465152,0.533333,1.0,UTA,2018-11-16
410,0.0,0.363636,0.442623,0.339713,0.444444,0.366667,0.477361,0.431818,0.444444,0.379310,...,0.055867,0.083533,0.349057,0.123248,0.343771,0.453030,0.533333,1.0,POR,2018-11-16
412,0.0,0.545455,0.475410,0.526316,0.074074,0.166667,0.169470,0.227273,0.314815,0.413793,...,0.053800,0.081400,0.375262,0.114872,0.346465,0.514394,0.400000,0.0,BOS,2018-11-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15171,0.0,0.386364,0.311475,0.464115,0.555556,0.600000,0.389392,0.295455,0.259259,0.206897,...,0.039400,0.084067,0.461076,0.168462,0.411111,0.572727,0.466667,0.0,DAL,2024-06-14
15172,0.0,0.181818,0.278689,0.224880,0.444444,0.516667,0.351876,0.272727,0.240741,0.137931,...,0.039400,0.090267,0.461076,0.176068,0.412121,0.550000,0.466667,1.0,DAL,2024-06-17
15173,0.0,0.568182,0.459016,0.564593,0.481481,0.450000,0.434670,0.340909,0.407407,0.448276,...,0.050800,0.089800,0.248358,0.214188,0.407744,0.533333,0.533333,0.0,BOS,2024-06-17
15174,0.0,0.386364,0.426230,0.377990,0.407407,0.483333,0.341527,0.386364,0.370370,0.517241,...,0.039800,0.090533,0.463242,0.197863,0.433670,0.559848,0.400000,1.0,NYK,2024-10-22


In [638]:
selected_columns = full.columns[~full.columns.isin(removed_columns)]

In [639]:
sfs.fit(full[selected_columns], full["target"])

SequentialFeatureSelector(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                          estimator=RidgeClassifier(alpha=1),
                          n_features_to_select=30)

In [640]:
predictors = list(selected_columns[sfs.get_support()])

In [641]:
predictions = backtest(full,rr,predictors)

In [642]:
accuracy_score(predictions["actual"], predictions["prediction"])

0.6344843786704252